In [1]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu
!pip install -q langchain langchain-community
!pip install jq
!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 6.8 MB/s eta 0:00:00


In [7]:
from huggingface_hub import notebook_login
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale

locale.getpreferredencoding = lambda: "UTF-8"
notebook_login()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import warnings
warnings.filterwarnings('ignore')

# import vector_database as vd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from model import HuggingFaceLLM
from pprint import pprint
from langchain.chains import LLMChain, SimpleSequentialChain

PINK = '\033[95m'
CYAN = '\033[96m'
YELLOW = '\033[93m'
NEON_GREEN = '\033[92m'
RESET_COLOR = '\033[0m'

# Load the embedding model
encoder_model_name = 'avsolatorio/GIST-small-Embedding-v0'
encoder_model = HuggingFaceEmbeddings(model_name=encoder_model_name)
vector_db = FAISS.load_local('/content/drive/MyDrive/dataset_mediassist/faiss_idx', encoder_model, allow_dangerous_deserialization=True)
retriever = vector_db.as_retriever(search_kwargs={'k': 1})

# Prompt template

# llm = ChatOllama(model="qwen2:0.5b")





In [21]:
llm = HuggingFaceLLM("meta-llama/Llama-2-7b-chat-hf", temperature = 0.1).llm



`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:


def start_chat():
    print("Starting conversation loop...")
    prompt_template_text = """
    <s>[INST] <<SYS>>
    You are a medical assistant. Given the information about the diagnosed disease in the CONTEXT, your task is to write a response justifying the diagnosis by explaining how the symptoms described by the USER align with those of the disease mentioned in the CONTEXT. The response should be in a very professional, first-person format. At the end, advise the user that if they do not feel the diagnosis is correct, they should provide a more concise description of their symptoms.

    CONTEXT:{context}
    <</SYS>>
    USER: {symptoms}
    [/INST]
    Assistant:
    """

    user_input = input(YELLOW + "Hello! I'm your personal Healthcare Diagnosis ChatBot.\nTell me about your symptoms:- " + RESET_COLOR)
    # user_input = message

    # parser = JsonOutputParser(pydantic_object=Disease)
    parser = StrOutputParser()

    prompt = PromptTemplate(
        input_variables=["context", "symptoms"],
        # partial_variables={"format_instructions": parser.get_format_instructions()},
        template=prompt_template_text,
    )


    retrived_docs = retriever.invoke(user_input)
    context = '\n'.join([doc.metadata['content'] for doc in retrived_docs])
    disease_name = retrived_docs[0].metadata['disease_name']
    generated_prompt = prompt.format(context=context, symptoms=user_input)


    chain_one = LLMChain(llm = llm, prompt = prompt)

    result = chain_one.run({"context": context, "symptoms": user_input})
    result = parser.invoke(result).split('Assistant:')[-1]



    while True:
        continue_chat(llm, disease_name, encoder_model_name)



In [49]:

def continue_chat(llm, disease_name, encoder_model_name):
    print(CYAN+"You can ask further questions regarding "+disease_name)
    prompt_template_text = """
    <s>[INST]<<SYS>>
    You are an expert in the health domain. Given a QUERY and CONTEXT, you should use your expertise and the provided CONTEXT to answer the QUERY. Ensure that your response is in the first person and begins in a way that clearly addresses the user's question.
    If the USER Query does not correspond with the CONTEXT, respond politely stating that their question doesn't relate to our current discussion, and unfortunately, I can't provide an answer.
    CONTEXT: {context}
    <</SYS>>
    USER: {question}
    [/INST]
    Assistant:
    """

    encoder_model = HuggingFaceEmbeddings(model_name=encoder_model_name)
    vector_db = FAISS.load_local(f'/content/drive/MyDrive/dataset_mediassist/disease_faiss_index/{disease_name}_faiss_idx', encoder_model, allow_dangerous_deserialization=True)
    retriever = vector_db.as_retriever(search_kwargs={'k': 5})
    user_input = input(PINK + "User:- " + RESET_COLOR)

    parser = StrOutputParser()

    # Define the prompt template
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template_text,
    )

    # Retrieve context documents based on user input
    context_docs = retriever.invoke(user_input)
    context_docs = [doc.page_content for doc in context_docs]
    context = '\n'.join(context_docs)


    generated_prompt = prompt.format(context=context, question=user_input)

    llm_chain = prompt | llm | parser
    # Invoke the chain with the context and user input
    result = llm_chain.invoke({"context": context, "question": user_input}).split('Assistant:')[-1]
    # print(NEON_GREEN + result + RESET_COLOR)
    return result

# if __name__ == "__main__":
#     main()
# # # # I have been experiencing often has no symptoms, severe hypertension can cause headaches, nosebleeds.","I have been experiencing often has no symptoms, severe hypertension can cause headaches, nosebleeds.


# while True:
#     continue_chat(llm, 'Diabetes', encoder_model_name) .split('Assistant:')[-1]

In [50]:
# continue_chat(llm, 'Kidney Stones', encoder_model_name)

In [51]:
!pip install --upgrade gradio

In [54]:
main()

<generator object main at 0x78417253f680>

In [52]:
import gradio as gr

gr.ChatInterface(main).launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://afb0926bbc467904c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --upgrade gradio